In [1]:
!pip install visdom
!pip install open3d
!pip install ninja
if True:
  !mkdir /content/repo_folder
  !rm -rf /content/MSN-Point-Cloud-Completion
  %cd /content/repo_folder
  !git clone https://github.com/iwillnotkillyou/MSN-Point-Cloud-Completion.git
  %cd ./MSN-Point-Cloud-Completion
  !git checkout Pavel
!git clone https://github.com/alexandrosstergiou/SoftPool.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408194 sha256=227894c29bc84f5afffe783539d581c23e164d716e497b8aa1507b59e7293df5
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextensio

In [ ]:
from pathlib import Path
import os
import torch
olf = "SoftPool/pytorch/CUDA"
newf = "softpool_cuda"
tempn = "tempnamed"
os.rename(newf, tempn)
!cp -r $olf ./
os.rename("CUDA", newf)
n = "soft_pool_cuda_module.py"
copiedfn = str(Path(tempn) / n)
destfn = str(Path(newf) / n)
!cp $copiedfn $destfn
!rm -r "CUDA"
!rm -r $tempn
from softpool_cuda.soft_pool_cuda_module import soft_pool3d, SoftPool3d
r = SoftPool3d()(torch.ones(100,3,100,100,100))
print(r.shape)

rm: cannot remove 'CUDA': No such file or directory
torch.Size([100, 3, 50, 50, 50])


In [ ]:
import os
dataf = "/content/repo_folder/MSN-Point-Cloud-Completion/data"
if True:
  from google.colab import drive
  drive.mount('/content/drive')
  drivef = "/content/drive/MyDrive/"
  !unzip -o /content/drive/MyDrive/val.zip -d $dataf
  !unzip -o /content/drive/MyDrive/complete.zip -d $dataf
  listp = f'{dataf}/train.list'
  valp = f'{dataf}/val.list'
  listop = f'{drivef}/train.list'
  valop = f'{drivef}/val.list'
  !cp $listop $listp
  !cp $valop $valp

Streaming output truncated to the last 5000 lines.
  inflating: /content/repo_folder/MSN-Point-Cloud-Completion/data/complete/02691156_50e6ab918250ac797a663d343339f8e.pcd  
  inflating: /content/repo_folder/MSN-Point-Cloud-Completion/data/complete/03001627_61b76525022f44fa9f2fd5a6f642de3d.pcd  
  inflating: /content/repo_folder/MSN-Point-Cloud-Completion/data/complete/03636649_ec876529d01d4df490ad276cd2af3a4.pcd  
  inflating: /content/repo_folder/MSN-Point-Cloud-Completion/data/complete/02691156_e7c6e2f7c5fd372e8e9091d822257500.pcd  
  inflating: /content/repo_folder/MSN-Point-Cloud-Completion/data/complete/04379243_6b678921222e3375f9395f1ee40f845e.pcd  
  inflating: /content/repo_folder/MSN-Point-Cloud-Completion/data/complete/04379243_e5c44394471f29d874739c2c9ecfb3ee.pcd  
  inflating: /content/repo_folder/MSN-Point-Cloud-Completion/data/complete/02933112_22e77a41cc388839824662341ce2b233.pcd  
  inflating: /content/repo_folder/MSN-Point-Cloud-Completion/data/complete/02691156_ee30cf

In [ ]:
if False:
  dataf = "/content/repo_folder/MSN-Point-Cloud-Completion/data"
  print('Downloading ...')
  !wget http://kaldir.vc.in.tum.de/cdiller/ShapeNetPointClouds.zip -P $dataf
  print('Extracting ...')
  fp = dataf + '/ShapeNetPointClouds.zip'
  !unzip -o -q $fp -d $dataf
  !rm $fp
  print('Done.')

In [ ]:
if False:
  dataf = "/content/repo_folder/MSN-Point-Cloud-Completion/data"
  fp = dataf + '/dataset.zip'
  !wget https://cloud.tsinghua.edu.cn/f/06a3c383dc474179b97d/?dl=1 -O $fp
  !unzip -o -q $fp -d $dataf

In [ ]:
%cd /content/repo_folder/MSN-Point-Cloud-Completion
!mkdir "/content/repo_folder/MSN-Point-Cloud-Completion/trained_model"
!cp "/content/drive/MyDrive/network.pth" "/content/repo_folder/MSN-Point-Cloud-Completion/trained_model/network.pth"
if False:
  !python val.py

/content/repo_folder/MSN-Point-Cloud-Completion


In [ ]:
import torch
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import os
import random
#from utils import *

def resample_pcd(pcd, n):
    """Drop or duplicate points so that pcd has exactly n points"""
    idx = np.random.permutation(pcd.shape[0])
    if idx.shape[0] < n:
        idx = np.concatenate([idx, np.random.randint(pcd.shape[0], size = n - pcd.shape[0])])
    return pcd[idx[:n]]

class ShapeNet(data.Dataset):
    def __init__(self, train = True, npoints = 8192, perc_data = 1):
        self.npoints = npoints
        self.train = train
        f = "./data/val/"
        files = os.listdir(f)
        self.model_list = ['_'.join(x.split('_')[:-2]) for x in files]
        split_ind = int(len(self.model_list)*perc_data*0.2)
        end = int(len(self.model_list)*perc_data)
        print(end)
        if train:
          self.model_list = self.model_list[split_ind:end]
        else:
          self.model_list = self.model_list[:split_ind]
        random.shuffle(self.model_list)
        self.len = len(self.model_list * 50)

    def __getitem__(self, index):
        model_id = self.model_list[index // 50]
        scan_id = index % 50
        def read_pcd(filename):
            pcd = o3d.io.read_point_cloud(filename)
            return torch.from_numpy(np.array(pcd.points)).float()
        partial = read_pcd(os.path.join("./data/val/", model_id + '_%d_denoised.pcd' % scan_id))
        complete = read_pcd(os.path.join("./data/complete/", '%s.pcd' % model_id))
        return model_id, resample_pcd(partial, 5000), resample_pcd(complete, self.npoints)

    def __len__(self):
        return self.len

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import sys
import expansion_penalty.expansion_penalty_module as expansion
import MDS.MDS_module as MDS_module
from model import *

class LocallyConnected1d():
    def __init__(self, in_channels,out_channels,output_size, bias = False, kernel_size = 1, stride = 1):
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.weight = nn.Parameter(
            torch.randn(output_size, out_channels, in_channels * kernel_size)
        )
        if bias:
            self.bias = nn.Parameter(
                torch.randn(out_channels, output_size)
            )
        else:
            self.register_parameter('bias', None)
    def forward(self,x):
        x = x.unfold(1, self.kernel_size, self.stride)
        print(x.shape)
        out = torch.matmul(x.unsqueeze(2), self.weight).squeeze()
        if self.bias is not None:
            out += self.bias
        return out

class SimpleLocallyConnected1d():
    def __init__(self, in_channels,out_channels,output_size, bias = False):
        super().__init__()
        self.weight = nn.Parameter(
            torch.randn(output_size, out_channels, in_channels)
        )
        if bias:
            self.bias = nn.Parameter(
                torch.randn(output_size, out_channels)
            )
        else:
            self.register_parameter('bias', None)
    def forward(self,x):
        out = torch.matmul(x.unsqueeze(2),self.weight).squeeze()
        if self.bias is not None:
            out += self.bias
        return out

class ResSmall(nn.Module):
    def __init__(self, res):
        super().__init__()
        self.conv1 = torch.nn.Conv1d(4, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 512, 1)
        self.conv4 = torch.nn.Conv1d(576, 256, 1)
        self.conv5 = torch.nn.Conv1d(256, 128, 1)
        self.conv6 = torch.nn.Conv1d(128, 3, 1)


        self.bn1 = torch.nn.BatchNorm1d(64)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.bn3 = torch.nn.BatchNorm1d(512)
        self.bn4 = torch.nn.BatchNorm1d(256)
        self.bn5 = torch.nn.BatchNorm1d(128)
        self.bn6 = torch.nn.BatchNorm1d(3)
        self.th = nn.Tanh()

    def forward(self, x):
        batchsize = x.size()[0]
        npoints = x.size()[2]
        x = F.relu(self.bn1(self.conv1(x)))
        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x,_ = torch.max(x, 2)
        x = x.view(-1, 512)
        x = x.view(-1, 512, 1).repeat(1, 1, npoints)
        x = torch.cat([x, pointfeat], 1)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.th(self.bn6(self.conv6(x)))
        return x

class BatchNormLocalConv1D(nn.Module):
    def __init__(self, in_channels,out_channels,output_size,kernel_size = 1, stride = 1):
        super().__init__()
        self.conv = LocallyConnected1d(in_channels,out_channels,output_size,False,kernel_size,stride)
        self.bn = torch.nn.BatchNorm1d(out_channels)

    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))

class BatchNormConv1D(nn.Module):
    def __init__(self, in_channels,out_channels,kernel_size = 1, stride = 1):
        super(BatchNormConv1D, self).__init__()
        self.conv = nn.Conv1d(in_channels,out_channels,kernel_size,stride)
        self.bn = torch.nn.BatchNorm1d(out_channels)

    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))

def make(sizes,f,additional_args = None):
    return [f(*([sizes[i],sizes[i+1]]+ [] if additional_args is None or additional_args[i] is None else additional_args[i])) for i in range(len(sizes)-1)]

from softpool_cuda.soft_pool_cuda_module import soft_pool1d, SoftPool1d

class GlobalTransform(nn.Module):
    def __init__(self, bottleneck_size, decoded_size, out_dim, latents):
        super().__init__()
        self.bottleneck_size = bottleneck_size
        self.decoded_size = decoded_size
        self.latents = latents
        sizes0 = [decoded_size, self.latents]
        self.convs0 = nn.Sequential(*make(sizes0, lambda x,y : nn.Sequential(BatchNormConv1D(x,y))))
        sizes1 = [self.latents, self.latents*self.latents]
        sizes2 = [self.latents, out_dim]
        self.convs1 = nn.Sequential(*make(sizes1, lambda x,y : nn.Sequential(BatchNormConv1D(x,y), nn.MaxPool1d(2))))
        self.convs2 = nn.Sequential(*make(sizes2, lambda x,y : nn.Sequential(BatchNormConv1D(x,y))))
        self.register_buffer('identity', torch.diag(torch.ones(self.latents)))


    def forward(self, partial, bottlenecked, decoded):
        bs = decoded.shape[0]
        x = self.convs0(decoded)
        transform_pre = self.convs1(x)
        softmaxweights = F.softmax(transform_pre,2)
        transform = (softmaxweights*transform_pre).sum(2).view(-1, self.latents, self.latents).contiguous()
        identity = torch.broadcast_to(self.identity.unsqueeze(0),(bs, self.identity.shape[0], self.identity.shape[1]))
        transform = transform + identity
        x = torch.matmul(transform, x.transpose(0, 1).transpose(0,2).unsqueeze(3))
        x = x.squeeze().transpose(0, 2).transpose(0, 1).contiguous()
        return self.convs2(x)

class GlobalTransformWLocalConnections(nn.Module):
    def __init__(self, bottleneck_size, decoded_size, out_dim, latents, n_primitives):
        super().__init__()
        self.bottleneck_size = bottleneck_size
        self.decoded_size = decoded_size
        self.latents = latents
        sizes0 = [decoded_size, self.latents]
        self.convs0 = nn.Sequential(*make(sizes0, lambda x,y : nn.Sequential(BatchNormConv1D(x,y))))
        sizes1 = [n_primitives, self.latents*self.latents]
        sizes2 = [self.latents, out_dim]
        GToutsize = self.latents
        self.convs = nn.Sequential(*[GlobalTransform(bottleneck_size,self.latents,GToutsize,self.latents) for x in range(n_primitives)])
        self.convs_transform = nn.Sequential(*make(sizes1, lambda x,y : BatchNormLocalConv1D(x,y,GToutsize,2,1)))
        self.convs2 = nn.Sequential(*make(sizes2, lambda x,y : nn.Sequential(BatchNormConv1D(x,y))))
        self.register_buffer('identity', torch.diag(torch.ones(self.latents)))


    def forward(self, partial, bottlenecked, decoded):
        bs = decoded.shape[0]
        x = self.convs0(decoded)
        vs = []
        for conv in self.convs:
            vs.append(conv(x))
        transform_pre = self.convs_transform(torch.transpose(torch.cat(vs,1),1,2).contiguous())
        softmaxweights = F.softmax(transform_pre,2)
        identity = self.identity.repeat(bs, 1)
        transform = (softmaxweights*transform_pre).view(-1, self.latents, self.latents).contiguous() + identity
        x = torch.matmul(transform, x.transpose(0, 1).transpose(0,2).unsqueeze(3))
        x = x.squeeze().transpose(0, 2).transpose(0, 1).contiguous()
        return self.convs2(x)

class TransformMSEmbedder(nn.Module):
  def __init__(self):
      super().__init__()
      self.encoder = nn.Sequential(
            PointNetfeat(num_points, global_feat=True),
            nn.Linear(1024, self.bottleneck_size),
            nn.BatchNorm1d(self.bottleneck_size),
            nn.ReLU())
      self.decoder = nn.ModuleList(
          [PointGenCon(bottleneck_size=2 + self.bottleneck_size) for i in range(0, self.n_primitives)])
      for de in self.decoder:
        de.conv4 = nn.Identity()
        de.th = nn.Identity()

  def freeze(self):
      for param in self.parameters():
          param.requires_grad = False

  def forward(self):
      partial = x
      x = self.encoder(x)
      outs = []
      for i in range(0, self.n_primitives):
          rand_grid = Variable(torch.cuda.FloatTensor(x.size(0), 2, self.num_points // self.n_primitives))
          rand_grid.data.uniform_(0, 1)
          y = x.unsqueeze(2).expand(x.size(0), x.size(1), rand_grid.size(2)).contiguous()
          y = torch.cat((rand_grid, y), 1).contiguous()
          outs.append(self.decoder[i](y))
      return torch.cat(outs, 2)

class TransformMSNNoEmbedder(nn.Module):
  def __init__(self, residualFunc = lambda x : x,
                 gtFunc = lambda x,y,z : GlobalTransform(x, y, z, 50),
                 num_points=8192, bottleneck_size=1024, n_primitives=16):
        super().__init__()
        self.num_points = num_points
        self.bottleneck_size = bottleneck_size
        self.n_primitives = n_primitives
        print(self.decoder[0].conv3.out_channels)
        self.global_transform = gtFunc(2 + self.bottleneck_size, self.decoder[0].conv3.out_channels, 3)
        self.res = PointNetRes()
        self.residual = residualFunc(self.res)
        self.expansion = expansion.expansionPenaltyModule()

  def forward(self, x):
      outs = self.global_transform(None,None,x).contiguous()
      out1 = outs.transpose(1, 2).contiguous()
      dist, _, mean_mst_dis = self.expansion(out1, self.num_points // self.n_primitives, 1.5)
      loss_mst = torch.mean(dist)

      id0 = torch.zeros(outs.shape[0], 1, outs.shape[2], device=torch.device('cuda')).contiguous()
      outs = torch.cat((outs, id0), 1)
      id1 = torch.ones(partial.shape[0], 1, partial.shape[2], device=torch.device('cuda')).contiguous()
      partial = torch.cat((partial, id1), 1)
      xx = torch.cat((outs, partial), 2)

      resampled_idx = MDS_module.minimum_density_sample(xx[:, 0:3, :].transpose(1, 2).contiguous(), out1.shape[1],mean_mst_dis)
      xx = MDS_module.gather_operation(xx, resampled_idx)
      delta = self.residual(xx)
      xx = xx[:, 0:3, :]
      out2 = (xx + delta).transpose(2, 1).contiguous()
      return out1, out2, loss_mst

class TransformMSN(nn.Module):
    def __init__(self, residualFunc = lambda x : x,
                 gtFunc = lambda x,y,z : GlobalTransform(x, y, z, 50),
                 num_points=8192, bottleneck_size=1024, n_primitives=16):
        super().__init__()
        self.num_points = num_points
        self.bottleneck_size = bottleneck_size
        self.n_primitives = n_primitives
        self.encoder = nn.Sequential(
            PointNetfeat(num_points, global_feat=True),
            nn.Linear(1024, self.bottleneck_size),
            nn.BatchNorm1d(self.bottleneck_size),
            nn.ReLU()
        )
        self.decoder = nn.ModuleList(
            [PointGenCon(bottleneck_size=2 + self.bottleneck_size) for i in range(0, self.n_primitives)])
        for de in self.decoder:
          de.conv4 = nn.Identity()
          de.th = nn.Identity()
        print(self.decoder[0].conv3.out_channels)
        self.global_transform = gtFunc(2 + self.bottleneck_size, self.decoder[0].conv3.out_channels, 3)
        self.res = PointNetRes()
        self.residual = residualFunc(self.res)
        self.expansion = expansion.expansionPenaltyModule()

    def freeze(self):
        for param in self.encoder.parameters():
            param.requires_grad = False
        for param in self.decoder.parameters():
            param.requires_grad = False

    def forward(self, x):
        partial = x
        x = self.encoder(x)
        outs = []
        for i in range(0, self.n_primitives):
            rand_grid = Variable(torch.cuda.FloatTensor(x.size(0), 2, self.num_points // self.n_primitives))
            rand_grid.data.uniform_(0, 1)
            y = x.unsqueeze(2).expand(x.size(0), x.size(1), rand_grid.size(2)).contiguous()
            y = torch.cat((rand_grid, y), 1).contiguous()
            outs.append(self.decoder[i](y))


        outs = torch.cat(outs, 2)
        outs = self.global_transform(partial,x,outs).contiguous()
        out1 = outs.transpose(1, 2).contiguous()
        dist, _, mean_mst_dis = self.expansion(out1, self.num_points // self.n_primitives, 1.5)
        loss_mst = torch.mean(dist)

        id0 = torch.zeros(outs.shape[0], 1, outs.shape[2], device=torch.device('cuda')).contiguous()
        outs = torch.cat((outs, id0), 1)
        id1 = torch.ones(partial.shape[0], 1, partial.shape[2], device=torch.device('cuda')).contiguous()
        partial = torch.cat((partial, id1), 1)
        xx = torch.cat((outs, partial), 2)

        resampled_idx = MDS_module.minimum_density_sample(xx[:, 0:3, :].transpose(1, 2).contiguous(), out1.shape[1],mean_mst_dis)
        xx = MDS_module.gather_operation(xx, resampled_idx)
        delta = self.residual(xx)
        xx = xx[:, 0:3, :]
        out2 = (xx + delta).transpose(2, 1).contiguous()
        return out1, out2, loss_mst

In [ ]:
%cd /content/repo_folder/MSN-Point-Cloud-Completion
!git sync
%load_ext autoreload
%autoreload 2
import open3d as o3d
import argparse
import random
import numpy as np
import torch
import torch.optim as optim
import sys
from model import *
from utils import *
import os
import json
import time, datetime
from time import time
from torch.utils.cpp_extension import load
import emd.emd_module as emd

class FullModel(nn.Module):
    def __init__(self, model):
        super(FullModel, self).__init__()
        self.model = model
        self.EMD = emd.emdModule()

    def forward(self, inputs, gt, eps, iters):
        output1, output2, expansion_penalty = self.model(inputs)
        gt = gt[:, :, :3]

        dist, _ = self.EMD(output1, gt, eps, iters)
        emd1 = torch.sqrt(dist).mean(1).contiguous()

        dist, _ = self.EMD(output2, gt, eps, iters)
        emd2 = torch.sqrt(dist).mean(1).contiguous()

        return output1, output2, emd1, emd2, expansion_penalty

torch.cuda.empty_cache()
import gc
gc.collect()
class opt:
  def __init__(self):
    self.batchSize = 8
    self.workers = 1
    self.nepoch = 100
    self.epoch_iter_limit = 320
    self.model = ''
    self.num_points = 8192
    self.n_primitives = 16
    self.env = "MSN_TRAIN"
    self.perc_data = 1
    self.manualSeed = 0
    self.epoch_iter_limit_val = 16
default_opt = opt()

def make_model(args = default_opt):
  model = TransformMSN(num_points = args.num_points, n_primitives = args.n_primitives)
  network = FullModel(model)
  #network = torch.nn.DataParallel(network)
  print('modelf',args.model)
  #model.apply(weights_init) #initialization of the weight
  if args.model != '':
      previous_state_dict = torch.load(args.model)
      model.load_state_dict(previous_state_dict,False,True)
      print("Previous weight loaded ")
      if False:
        sd = model.state_dict()
        for w in sd:
          if w in previous_state_dict:
            print(w)
            print(torch.equal(sd[w], previous_state_dict[w]))
  model.freeze()
  network.cuda()
  return network

def train(network, dir_name, args = default_opt):
  lrate = 0.001 #learning rate
  optimizer = optim.Adam(model.parameters(), lr = lrate)
  train_curve = []
  val_curve = []
  labels_generated_points = (torch.Tensor(range(1, (args.n_primitives+1)*(args.num_points//args.n_primitives)+1))
  .view(args.num_points//args.n_primitives,(args.n_primitives+1)).transpose(0,1))
  labels_generated_points = (labels_generated_points)%(args.n_primitives+1)
  labels_generated_points = labels_generated_points.contiguous().view(-1)
  if not os.path.exists('./log/'):
      os.mkdir('./log/')
  if not os.path.exists(dir_name):
      os.mkdir(dir_name)
  logname = os.path.join(dir_name, 'log.txt')
  os.system('cp ./train.py %s' % dir_name)
  os.system('cp ./dataset.py %s' % dir_name)
  os.system('cp ./model.py %s' % dir_name)
  with open(logname, 'a') as f: #open and append
          f.write(str(model) + '\n')
  print("Random Seed: ", args.manualSeed)
  random.seed(args.manualSeed)
  torch.manual_seed(args.manualSeed)
  best_val_loss = 10

  dataset = ShapeNet(train=True, npoints=args.num_points, perc_data = args.perc_data)
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=args.batchSize,
                                            shuffle=True, num_workers = args.workers)
  dataset_test = ShapeNet(train=False, npoints=args.num_points, perc_data = args.perc_data)
  dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=args.batchSize,
                                            shuffle=False, num_workers = args.workers)

  len_dataset = len(dataset)
  print("Train Set Size: ", len_dataset)
  for epoch in range(args.nepoch):
      train_loss = np.zeros(len(dataloader))
      val_loss = np.zeros(len(dataloader))
      #TRAIN MODE
      model.train()

      # learning rate schedule
      if epoch==20:
          optimizer = optim.Adam(model.parameters(), lr = lrate/10.0)
      if epoch==40:
          optimizer = optim.Adam(model.parameters(), lr = lrate/100.0)

      for i, data in enumerate(dataloader, 0):
          if args.epoch_iter_limit is not None and i > args.epoch_iter_limit:
            break
          optimizer.zero_grad()
          id, input, gt = data
          input = input.float().cuda()
          gt = gt.float().cuda()
          input = input.transpose(2,1).contiguous()

          output1, output2, emd1, emd2, expansion_penalty  = network(input, gt.contiguous(), 0.005, 50)
          emd1m = emd1.mean()
          emd1mi = emd1m.item()
          emd2m = emd2.mean()
          emd2mi = emd2m.item()
          exppm = expansion_penalty.mean()
          exppmi = exppm.item()
          loss_net = emd1m + emd2m+ exppm * 0.1
          train_loss[i] = emd2mi
          loss_net.backward()
          optimizer.step()

          print(args.env + ' train [%d: %d/%d]  emd1: %f emd2: %f expansion_penalty: %f'
                %(epoch, i, len_dataset/args.batchSize, emd1mi, emd2mi,
                  exppmi))
      train_curve.append(np.mean(train_loss))

      # VALIDATION
      if epoch % 2 == 0:
          model.eval()
          with torch.no_grad():
              for i, data in enumerate(dataloader_test, 0):
                  if args.epoch_iter_limit_val is not None and i > args.epoch_iter_limit_val:
                    break
                  id, input, gt = data
                  input = input.float().cuda()
                  gt = gt.float().cuda()
                  input = input.transpose(2,1).contiguous()
                  output1, output2, emd1, emd2, expansion_penalty  = network(input, gt.contiguous(), 0.004, 3000)
                  emd1m = emd1.mean()
                  emd1mi = emd1m.item()
                  emd2m = emd2.mean()
                  emd2mi = emd2m.item()
                  val_loss[i] = emd2mi
                  exppm = expansion_penalty.mean()
                  exppmi = exppm.item()
                  idx = random.randint(0, input.size()[0] - 1)
                  print(args.env + ' val [%d: %d/%d]  emd1: %f emd2: %f expansion_penalty: %f'
                        %(epoch, i, len_dataset/args.batchSize, emd1mi,
                          emd2mi, exppmi))

      val_curve.append(np.mean(val_loss))
      log_table = {
        "train_loss" : np.mean(train_loss),
        "val_loss" : np.mean(val_loss),
        "epoch" : epoch,
        "lr" : lrate,
        "bestval" : best_val_loss,

      }
      with open(logname, 'a') as f:
          f.write('json_stats: ' + json.dumps(log_table) + '\n')

      print('saving net...')
      torch.save(model.state_dict(), '%s/network.pth' % (dir_name))

def cm1(model):
  for n, m in model.named_children():
    print(n,m)
  return model

now = datetime.datetime.now()
save_path = "/content/drive/MyDrive/models" + now.isoformat()
default_opt.manualSeed = random.randint(1, 10000)
default_opt.model = './trained_model/network.pth'
print(default_opt.model)
model = make_model(default_opt)
cs = 0
for name, param in model.named_parameters():
  if param.requires_grad:
    cs += param.nelement()
    print(name, cs)


if True:
  train(model,save_path, default_opt)

/content/repo_folder/MSN-Point-Cloud-Completion
git: 'sync' is not a git command. See 'git --help'.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
./trained_model/network.pth
256
modelf ./trained_model/network.pth
Previous weight loaded 
model.global_transform.convs0.0.0.conv.weight 12800
model.global_transform.convs0.0.0.conv.bias 12850
model.global_transform.convs0.0.0.bn.weight 12900
model.global_transform.convs0.0.0.bn.bias 12950
model.global_transform.convs1.0.0.conv.weight 137950
model.global_transform.convs1.0.0.conv.bias 140450
model.global_transform.convs1.0.0.bn.weight 142950
model.global_transform.convs1.0.0.bn.bias 145450
model.global_transform.convs2.0.0.conv.weight 145600
model.global_transform.convs2.0.0.conv.bias 145603
model.global_transform.convs2.0.0.bn.weight 145606
model.global_transform.convs2.0.0.bn.bias 145609
model.res.conv1.weight 145865
model.res.conv1.bias 145929
model.res.conv2.weight 154121
model.res.conv2.bias 1542

KeyboardInterrupt: 